In [554]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier

In [163]:
songDf = pd.read_csv("songData.csv")

In [164]:
songDf.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'acousticness', u'analysis_url',
       u'danceability', u'duration_ms', u'energy', u'id', u'instrumentalness',
       u'key', u'liveness', u'loudness', u'mode', u'speechiness', u'tempo',
       u'time_signature', u'track_href', u'type', u'uri', u'valence', u'genre',
       u'artist', u'date', u'name', u'popularity', u'lyric'],
      dtype='object')

In [165]:
songDf = songDf.drop(['Unnamed: 0', 'Unnamed: 0.1', 'analysis_url', 'id','track_href','type', 'uri', 'artist', 'date'
                     ,'name', 'popularity', 'lyric'], axis = 1)

In [166]:
songDf = songDf.sample(frac = 1, replace = False)

In [167]:
songDf = pd.get_dummies(songDf, columns=['key','time_signature'])

In [168]:
songDf.columns

Index([u'acousticness', u'danceability', u'duration_ms', u'energy',
       u'instrumentalness', u'liveness', u'loudness', u'mode', u'speechiness',
       u'tempo', u'valence', u'genre', u'key_0', u'key_1', u'key_2', u'key_3',
       u'key_4', u'key_5', u'key_6', u'key_7', u'key_8', u'key_9', u'key_10',
       u'key_11', u'time_signature_0', u'time_signature_1',
       u'time_signature_3', u'time_signature_4', u'time_signature_5'],
      dtype='object')

In [169]:
train = songDf.iloc[0:8000]

In [170]:
test = songDf.iloc[8000:11240]

In [171]:
trainLabels = train['genre'].astype('category')
testLabels = test['genre'].astype('category')

In [172]:
train = train.drop(['genre'], axis = 1)
test = test.drop(['genre'], axis = 1)

In [173]:
train = preprocessing.scale(train)
test = preprocessing.scale(test)

In [271]:
def fold(model, trainData, labels, n_folds, n_class):
    accs = 0
    precs = np.array([0]*n_class)
    recalls = np.array([0]*n_class)
    
    
    
    for x in range(0,n_folds):
        start = x * len(trainData)/n_folds
        end = (x+1) * len(trainData)/n_folds
        index = range(0,start) + range(end,len(trainData))
        fit = model.fit(X = trainData[index], y = labels.iloc[index] )
        score = fit.score(trainData[start:end], labels.iloc[start:end])
        accs = accs + score
        predictions = fit.predict(trainData[start:end])
        true = labels.iloc[start:end]
        prf = precision_recall_fscore_support(y_true = true, y_pred = predictions)
        precs = precs + prf[0]
        recalls = (recalls + prf[1])
    
    accs = accs/float(n_folds)
    precs = precs/float(n_folds)
    recalls = recalls/float(n_folds)
    
    return((accs, precs, recalls, fit.classes_))
    

In [ ]:
### multinomial logistic regression with balanced weights

lr = LogisticRegression(multi_class = "multinomial", solver = "saga" , class_weight = "balanced")

In [272]:
lrAccs = fold(lr, train, trainLabels, 4, 8)
lrAccs

(0.67225000000000001,
 array([ 0.43888155,  0.32304214,  0.84701375,  0.52970877,  0.84692475,
         0.2606726 ,  0.87934448,  0.63538098]),
 array([ 0.56176853,  0.72665692,  0.88365781,  0.68380031,  0.83375188,
         0.50002792,  0.70648906,  0.3950369 ]),
 array(['blues', 'country', 'electric', 'jazz', 'metal', 'pop', 'rap',
        'rock'], dtype=object))

In [273]:
### multinomial logistic regression without balanced weights

lr = LogisticRegression(multi_class = "multinomial", solver = "saga")

In [274]:
lrAccs = fold(lr, train, trainLabels, 4, 8)
lrAccs

(0.72450000000000003,
 array([ 0.53619479,  0.48578935,  0.9048136 ,  0.57673724,  0.82939207,
         0.42559524,  0.78219848,  0.5694021 ]),
 array([ 0.44272304,  0.18576998,  0.83542707,  0.62041421,  0.87131811,
         0.06739966,  0.88875885,  0.66232911]),
 array(['blues', 'country', 'electric', 'jazz', 'metal', 'pop', 'rap',
        'rock'], dtype=object))

In [275]:
#### KNN using default of 5 neighbours
knn = KNeighborsClassifier()

In [277]:
knnAccs = fold(knn, train, trainLabels, 4, 8)
knnAccs

(0.65449999999999997,
 array([ 0.42678646,  0.31848419,  0.82962018,  0.57592482,  0.74955692,
         0.24173675,  0.76680284,  0.54543389]),
 array([ 0.37670438,  0.40645712,  0.80833525,  0.54497908,  0.81453481,
         0.18030279,  0.77747235,  0.49675576]),
 array(['blues', 'country', 'electric', 'jazz', 'metal', 'pop', 'rap',
        'rock'], dtype=object))

In [293]:
#### Random Forest

rf = RandomForestClassifier(n_estimators = 50, max_depth = 8, class_weight = "balanced")

In [294]:
rfAccs = fold(rf, train, trainLabels, 4, 8)
rfAccs

(0.74012500000000003,
 array([ 0.4507556 ,  0.44976838,  0.91222538,  0.54324125,  0.87425009,
         0.34000982,  0.86783683,  0.72433161]),
 array([ 0.61602249,  0.82838694,  0.93874308,  0.67126889,  0.88320856,
         0.42540423,  0.81933855,  0.48595493]),
 array(['blues', 'country', 'electric', 'jazz', 'metal', 'pop', 'rap',
        'rock'], dtype=object))

In [242]:
rf.fit(train, trainLabels).feature_importances_

array([  1.45263800e-01,   1.33974868e-01,   8.32726935e-02,
         1.07763684e-01,   1.17891742e-01,   4.03268048e-02,
         6.31114198e-02,   9.63086076e-03,   1.06034649e-01,
         5.69270281e-02,   7.75708739e-02,   4.29383022e-03,
         5.75287580e-03,   4.88668739e-03,   2.29640577e-03,
         3.97694907e-03,   3.85665771e-03,   3.66517526e-03,
         4.83100028e-03,   3.66754787e-03,   4.46174482e-03,
         3.35122175e-03,   3.75124340e-03,   6.29198652e-05,
         4.38015672e-04,   3.76157991e-03,   4.11588696e-03,
         1.06183433e-03])

In [243]:
songDf.columns

Index([u'acousticness', u'danceability', u'duration_ms', u'energy',
       u'instrumentalness', u'liveness', u'loudness', u'mode', u'speechiness',
       u'tempo', u'valence', u'genre', u'key_0', u'key_1', u'key_2', u'key_3',
       u'key_4', u'key_5', u'key_6', u'key_7', u'key_8', u'key_9', u'key_10',
       u'key_11', u'time_signature_0', u'time_signature_1',
       u'time_signature_3', u'time_signature_4', u'time_signature_5'],
      dtype='object')

In [288]:
### SVM one vs all

sv = SVC(kernel = "rbf", class_weight = "balanced")
fold(sv, train, trainLabels, 4, 8)


(0.68037499999999995,
 array([ 0.40712671,  0.33140727,  0.86707079,  0.52556799,  0.87265146,
         0.27743175,  0.88437632,  0.63944702]),
 array([ 0.56199191,  0.72266082,  0.89539911,  0.66513737,  0.81083581,
         0.51680307,  0.710707  ,  0.45015157]),
 array(['blues', 'country', 'electric', 'jazz', 'metal', 'pop', 'rap',
        'rock'], dtype=object))

In [298]:
### Gradient Boosting Classifier 

gbc = GradientBoostingClassifier(loss = "deviance")

In [299]:
fold(gbc, train, trainLabels, 4, 8)

(0.77712499999999995,
 array([ 0.57148811,  0.67859964,  0.95105359,  0.58169712,  0.88027143,
         0.45020212,  0.83250251,  0.65741802]),
 array([ 0.44694969,  0.56423002,  0.93714504,  0.61737358,  0.87503663,
         0.2279888 ,  0.90732538,  0.7189776 ]),
 array(['blues', 'country', 'electric', 'jazz', 'metal', 'pop', 'rap',
        'rock'], dtype=object))

In [556]:
mlnn = MLPClassifier(solver = "sgd", activation = "logistic")
fold(mlnn, train, trainLabels, 4, 8)

(0.70399999999999996,
 array([ 0.53033911,  0.        ,  0.89739798,  0.54859271,  0.79801235,
         0.        ,  0.76220407,  0.53451418]),
 array([ 0.3376839 ,  0.        ,  0.76544486,  0.61382595,  0.87259103,
         0.        ,  0.89775932,  0.66443158]),
 array(['blues', 'country', 'electric', 'jazz', 'metal', 'pop', 'rap',
        'rock'],
       dtype='|S8'))

In [464]:
####Now Going to add lyrics data [will have to censor certain songs bc no lyrics] - need to do imports
import collections, re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [465]:
####Creating Vector Matrix for our words
vectorizer = CountVectorizer(stop_words = 'english')
songDf = pd.read_csv("songData.csv")
emptyList = (pd.isnull(songDf['lyric']))
notEmptyList = list(~np.array(emptyList))
notEmptyIndex = np.where(notEmptyList)[0]
songDf_lyrics = songDf.iloc[notEmptyIndex]

data_corpus = songDf['lyric'][notEmptyIndex]
X = vectorizer.fit_transform(data_corpus) 
tfidfTransform = TfidfTransformer()
tfidfValues = tfidfTransform.fit_transform(X)
tfidfMatrix = pd.DataFrame(tfidfValues.toarray())
tfidfMatrix.columns = vectorizer.get_feature_names()

In [466]:
rapScore = (tfidfMatrix['nigga']+tfidfMatrix['niggas']+tfidfMatrix['money']+tfidfMatrix['shit']+tfidfMatrix['fuck']
 +tfidfMatrix['bitch']+tfidfMatrix['ain']+tfidfMatrix['hook']+tfidfMatrix['verse']) * 10

metalScore = (tfidfMatrix['life']+tfidfMatrix['hell']+tfidfMatrix['death']+tfidfMatrix['blood']+tfidfMatrix['world']
              +tfidfMatrix['dead']+tfidfMatrix['god']+tfidfMatrix['flesh']+tfidfMatrix['fear'] 
              +tfidfMatrix['god']+tfidfMatrix['pain']+tfidfMatrix['die']+tfidfMatrix['eyes']
              +tfidfMatrix['shall']+tfidfMatrix['end']) * 10

popScore = (tfidfMatrix['oh'] + tfidfMatrix['chorus']+tfidfMatrix['love']) * 10

rockScore = (tfidfMatrix['bridge'] + tfidfMatrix['love']+tfidfMatrix['cause']) * 10

countryScore = (tfidfMatrix['country']+tfidfMatrix['road'] + tfidfMatrix['chorus'] ) * 10

jazzScore = (tfidfMatrix['instrumental']+tfidfMatrix['jazz'] ) * 10

bluesScore = (tfidfMatrix['blues']+tfidfMatrix['instrumental'] ) * 10

In [467]:
#### Remove useless columns
songDf_lyrics = songDf_lyrics.drop(['Unnamed: 0', 'Unnamed: 0.1', 'analysis_url', 'id','track_href','type', 'uri', 'artist', 'date'
                     ,'name', 'popularity', 'lyric'], axis = 1)

In [468]:
songDf_lyrics.columns = ['acousticness_1', 'danceability_1', 'duration_ms_1', 'energy_1', 'instrumentalness_1', 'key_1',
                        'liveness_1', 'loudness_1', 'mode_1', 'speechiness_1', 'tempo_1', 'time_signature_1', 'valence_1', 
                         'genre_1']

In [469]:
#### Concatenate Dataframes
scoreData = pd.concat([songDf_lyrics.reset_index(), rapScore, metalScore, popScore, rockScore, countryScore, jazzScore, bluesScore], axis=1)
lyricsData = pd.concat([songDf_lyrics.reset_index(), tfidfMatrix], axis=1)

In [470]:
scoreData;

In [471]:
lyricsData = lyricsData.reset_index()

In [472]:
lyricsData = lyricsData.drop(['level_0', 'index'], axis = 1)
scoreData = scoreData.drop(['index'], axis = 1)

In [473]:
lyricsData = lyricsData.sample(frac = 1, replace = False)
lyricsData = pd.get_dummies(lyricsData, columns=['key_1','time_signature_1'])

scoreData = scoreData.sample(frac = 1, replace = False)
scoreData = pd.get_dummies(scoreData, columns=['key_1','time_signature_1'])

In [474]:
elecIndex = lyricsData[lyricsData.genre_1 == "electric"].index
elecIndex

Int64Index([5599, 5600, 5601], dtype='int64')

In [475]:
lyricsData = lyricsData.drop([5601, 5599, 5600], axis = 0)
scoreData = scoreData.drop([5601, 5599, 5600], axis = 0)

In [476]:
train_lyric = lyricsData.iloc[0:6000]
test_lyric = lyricsData.iloc[6000:8120]

train_score = scoreData.iloc[0:6000]
test_score = scoreData.iloc[6000:8120]

In [477]:
trainLyric_label = train_lyric.genre_1
testLyric_label = test_lyric.genre_1

trainScore_label = train_score.genre_1
testScore_label = test_score.genre_1


In [478]:
train_lyric = train_lyric.drop(['genre_1'], axis = 1)
test_lyric = test_lyric.drop(['genre_1'], axis = 1)

train_score = train_score.drop(['genre_1'], axis = 1)
test_score = test_score.drop(['genre_1'], axis = 1)


In [479]:
train_lyric = preprocessing.scale(train_lyric)
test_lyric = preprocessing.scale(test_lyric)

In [480]:
train_score = preprocessing.scale(train_score)
test_score = preprocessing.scale(test_score)

In [481]:
### multinomial logistic regression with balanced weights

lr = LogisticRegression(multi_class = "multinomial", solver = "saga", class_weight = "balanced" )



lrAccs = fold(lr, train_score, trainScore_label, 3, 7)
lrAccs


(0.72116666666666662,
 array([ 0.23025794,  0.40768899,  0.53622943,  0.88116222,  0.41108753,
         0.93917616,  0.64920771]),
 array([ 0.53541667,  0.71610493,  0.80489795,  0.85551775,  0.64962979,
         0.83919533,  0.42899458]),
 array(['blues', 'country', 'jazz', 'metal', 'pop', 'rap', 'rock'], dtype=object))

In [482]:
#### KNN using default of 5 neighbours
knn = KNeighborsClassifier()

knnAccs = fold(knn, train_score, trainScore_label, 3, 7)
knnAccs

(0.67149999999999999,
 array([ 0.18511905,  0.33114289,  0.60135647,  0.77703879,  0.29857181,
         0.83322025,  0.54264796]),
 array([ 0.07127193,  0.41733513,  0.60642207,  0.84133105,  0.2940199 ,
         0.82704336,  0.48224509]),
 array(['blues', 'country', 'jazz', 'metal', 'pop', 'rap', 'rock'], dtype=object))

In [484]:
#### Random Forest

rf = RandomForestClassifier(n_estimators = 50, max_depth = 8, class_weight = "balanced")

rfAccs = fold(rf, train_score, trainScore_label, 3, 7)
rfAccs

(0.78666666666666674,
 array([ 0.32775083,  0.53391354,  0.63365893,  0.91388297,  0.49153261,
         0.90756851,  0.73541965]),
 array([ 0.35153509,  0.77301427,  0.86367674,  0.88608332,  0.64483363,
         0.89407001,  0.59269133]),
 array(['blues', 'country', 'jazz', 'metal', 'pop', 'rap', 'rock'], dtype=object))

In [486]:
### SVM one vs all

sv = SVC(kernel = "rbf", class_weight = "balanced")
fold(sv, train_score, trainScore_label, 3, 7)

(0.72616666666666674,
 array([ 0.20982692,  0.39525849,  0.56744317,  0.89713117,  0.39939054,
         0.93297109,  0.64865806]),
 array([ 0.41239035,  0.67103977,  0.72345548,  0.83899297,  0.59095875,
         0.84203647,  0.5150547 ]),
 array(['blues', 'country', 'jazz', 'metal', 'pop', 'rap', 'rock'], dtype=object))

In [490]:
### Gradient Boosting Classifier 

gbc = GradientBoostingClassifier(loss = "deviance", max_depth = 3)
fold(gbc, train_score, trainScore_label, 3, 7)

(0.81016666666666659,
 array([ 0.44731109,  0.70283438,  0.75585006,  0.89915462,  0.58075489,
         0.89730239,  0.68713314]),
 array([ 0.21600877,  0.57471632,  0.76320174,  0.89823148,  0.45910921,
         0.93142736,  0.75238143]),
 array(['blues', 'country', 'jazz', 'metal', 'pop', 'rap', 'rock'], dtype=object))

In [552]:
### Seeing what happens on testing Data

### Just Spotify Data using RandomForest

rf = RandomForestClassifier(n_estimators = 50, max_depth = 8, class_weight = "balanced")
rfFit = rf.fit(train, trainLabels)


#### Just Spotify Data using GBC
gbc = GradientBoostingClassifier(loss = "deviance")
gbcFit = gbc.fit(train, trainLabels)

##### Just Spotify Data using Logisitc Regression
lr = LogisticRegression(multi_class = "multinomial", solver = "saga", class_weight = "balanced")
lrFit = lr.fit(train, trainLabels)

In [538]:
## Accuracy
print(rfFit.score(test, testLabels))
print(gbcFit.score(test, testLabels))
print(lrFit.score(test, testLabels))

0.7260909935
0.769421231817
0.673166202414


In [541]:
###Precision
print(precision_recall_fscore_support(y_true = testLabels, y_pred = rfFit.predict(test))[0])
print(precision_recall_fscore_support(y_true = testLabels, y_pred = gbcFit.predict(test))[0])
print(precision_recall_fscore_support(y_true = testLabels, y_pred = lrFit.predict(test))[0])
print(rfFit.classes_)

[ 0.45070423  0.38110749  0.90647482  0.51960784  0.88126362  0.34433962
  0.86431479  0.66512702]
[ 0.55445545  0.57541899  0.98360656  0.56578947  0.85876289  0.58064516
  0.80089989  0.66246057]
[ 0.3902439   0.32716049  0.79139073  0.54761905  0.87301587  0.2875
  0.89090909  0.60470588]
['blues' 'country' 'electric' 'jazz' 'metal' 'pop' 'rap' 'rock']


In [542]:
### Recall
print(precision_recall_fscore_support(y_true = testLabels, y_pred = rfFit.predict(test))[1])
print(precision_recall_fscore_support(y_true = testLabels, y_pred = gbcFit.predict(test))[1])
print(precision_recall_fscore_support(y_true = testLabels, y_pred = lrFit.predict(test))[1])
print(rfFit.classes_)

[ 0.41558442  0.83571429  0.92647059  0.77372263  0.87082885  0.4591195
  0.82512953  0.43113772]
[ 0.36363636  0.73571429  0.88235294  0.62773723  0.89666308  0.22641509
  0.92227979  0.62874251]
[ 0.51948052  0.75714286  0.87867647  0.67153285  0.82884822  0.57861635
  0.69818653  0.38473054]
['blues' 'country' 'electric' 'jazz' 'metal' 'pop' 'rap' 'rock']


In [553]:
confusion_matrix(testLabels, gbcFit.predict(test), gbcFit.classes_)

array([[ 55,   3,   0,  44,   2,   2,  13,  35],
       [  0, 103,   0,   0,   0,   2,   8,  27],
       [  0,   0, 240,   0,  16,   0,  10,   6],
       [ 30,   1,   0,  86,   0,   1,  10,   9],
       [  0,   2,   1,   0, 834,   0,   4,  88],
       [  2,  17,   0,   5,   0,  36,  86,  13],
       [  2,   5,   0,   3,   3,  12, 712,  35],
       [ 10,  48,   3,  15, 114,  10,  46, 422]])

In [544]:
### Seeing what happens on testing Data

### Lyric Score Data using RandomForest

rf = RandomForestClassifier(n_estimators = 50, max_depth = 8, class_weight = "balanced")
rfFit = rf.fit(train_score, trainScore_label)


#### Lyric Score Data using GBC
gbc = GradientBoostingClassifier(loss = "deviance")
gbcFit = gbc.fit(train_score, trainScore_label)

##### Lyric Score Data using Logisitc Regression
lr = LogisticRegression(multi_class = "multinomial", solver = "saga", class_weight = "balanced")
lrFit = lr.fit(train_score, trainScore_label)

In [545]:
## Accuracy
print(rfFit.score(test_score, testScore_label))
print(gbcFit.score(test_score, testScore_label))
print(lrFit.score(test_score, testScore_label))

0.71138403401
0.803967879074
0.744922059518


In [546]:
###Precision
print(precision_recall_fscore_support(y_true = testScore_label, y_pred = rfFit.predict(test_score))[0])
print(precision_recall_fscore_support(y_true = testScore_label, y_pred = gbcFit.predict(test_score))[0])
print(precision_recall_fscore_support(y_true = testScore_label, y_pred = lrFit.predict(test_score))[0])
print(rfFit.classes_)

[ 0.3220339   0.31481481  0.58510638  0.9625      0.42328042  0.87021631
  0.59593023]
[ 0.57142857  0.62790698  0.7         0.95341098  0.54455446  0.86990596
  0.67684022]
[ 0.3047619   0.37        0.58762887  0.91137371  0.38190955  0.94528302
  0.71197411]
['blues' 'country' 'jazz' 'metal' 'pop' 'rap' 'rock']


In [547]:
###Recall
print(precision_recall_fscore_support(y_true = testScore_label, y_pred = rfFit.predict(test_score))[1])
print(precision_recall_fscore_support(y_true = testScore_label, y_pred = gbcFit.predict(test_score))[1])
print(precision_recall_fscore_support(y_true = testScore_label, y_pred = lrFit.predict(test_score))[1])
print(rfFit.classes_)

[ 0.38        0.86734694  0.90163934  0.77110157  0.68376068  0.8760469
  0.41414141]
[ 0.24        0.82653061  0.80327869  0.81974249  0.47008547  0.92964824
  0.76161616]
[ 0.64        0.75510204  0.93442623  0.88268956  0.64957265  0.83919598
  0.44444444]
['blues' 'country' 'jazz' 'metal' 'pop' 'rap' 'rock']


In [551]:
confusion_matrix(testScore_label, gbcFit.predict(test_score), gbcFit.classes_)

array([[ 12,   0,   7,   0,   0,   7,  24],
       [  0,  81,   0,   0,   6,   2,   9],
       [  3,   1,  49,   0,   3,   3,   2],
       [  0,   2,   0, 573,   1,   5, 118],
       [  0,   8,   2,   0,  55,  36,  16],
       [  1,   2,   2,   0,  26, 555,  11],
       [  5,  35,  10,  28,  10,  30, 377]])

In [493]:
### Trying Principal Components Analysis

pca = PCA(n_components = 2).fit_transform(train_lyric)

In [494]:
pca_df = pd.DataFrame(pca)

In [495]:
pca_df['genre'] = trainLyric_label.reset_index()['genre_1']

In [496]:
pca_df.columns = ['x','y','genre']

In [499]:
pca_df.to_csv("pcaFile.csv")

In [524]:
### Trying Principal Components Analysis with 75 components


train_lyric_notValid = train_lyric[0:4000]
notValid_label = trainLyric_label[0:4000]
train_lyric_valid = train_lyric[4000:6005]
valid_label = trainLyric_label[4000:6005]

In [511]:
pca_notValid = PCA(n_components = 75).fit_transform(train_lyric_notValid)


In [517]:
pca_Valid = PCA(n_components = 75).fit_transform(np.concatenate((train_lyric_notValid, train_lyric_valid), axis = 0))

In [518]:
pca_notValid = preprocessing.scale(pca_notValid)
pca_Valid = preprocessing.scale(pca_Valid)


In [527]:
pca_Valid = pca_Valid[4000:6005]

In [528]:
### Trying some models

### First Multinomial Logistic Regression
lr = LogisticRegression(multi_class = "multinomial", solver = "saga", class_weight = "balanced")
fit = lr.fit(pca_notValid, notValid_label)

fit.score(pca_Valid, valid_label)

0.219

In [526]:
len(pca_Valid)

6000